# Les 10: Labo - Oplossingen

**Mathematical Foundations - IT & Artificial Intelligence**

---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from scipy.optimize import minimize

np.set_printoptions(precision=4, suppress=True)
np.random.seed(42)

print("Libraries geladen!")

---

## Oefening 1: Likelihood Berekenen - Oplossingen

In [ ]:
# Opdracht 1a - Bernoulli likelihood
data = np.array([1, 1, 0, 1, 1, 1, 0, 1, 1, 1])  # 8 keer succes

def bernoulli_likelihood(p, data):
    """L(p | data) = Π p^x_i * (1-p)^(1-x_i)"""
    likelihood = np.prod(p**data * (1-p)**(1-data))
    return likelihood

def bernoulli_log_likelihood(p, data):
    """log L(p | data) = Σ [x_i*log(p) + (1-x_i)*log(1-p)]"""
    ll = np.sum(data * np.log(p) + (1-data) * np.log(1-p))
    return ll

# Test voor p = 0.5 en p = 0.8
print(f"Data: {data}")
print(f"Aantal successen: {np.sum(data)} van {len(data)}")
print(f"\nLikelihood(p=0.5) = {bernoulli_likelihood(0.5, data):.6f}")
print(f"Likelihood(p=0.8) = {bernoulli_likelihood(0.8, data):.6f}")
print(f"\nLog-likelihood(p=0.5) = {bernoulli_log_likelihood(0.5, data):.4f}")
print(f"Log-likelihood(p=0.8) = {bernoulli_log_likelihood(0.8, data):.4f}")

In [ ]:
# Opdracht 1b - Plot likelihood
p_values = np.linspace(0.01, 0.99, 100)
likelihoods = [bernoulli_likelihood(p, data) for p in p_values]
log_likelihoods = [bernoulli_log_likelihood(p, data) for p in p_values]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Likelihood
axes[0].plot(p_values, likelihoods, 'b-', linewidth=2)
axes[0].axvline(x=0.8, color='red', linestyle='--', label='MLE = 0.8')
axes[0].set_xlabel('p')
axes[0].set_ylabel('L(p | data)')
axes[0].set_title('Likelihood')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Log-likelihood
axes[1].plot(p_values, log_likelihoods, 'b-', linewidth=2)
axes[1].axvline(x=0.8, color='red', linestyle='--', label='MLE = 0.8')
axes[1].set_xlabel('p')
axes[1].set_ylabel('log L(p | data)')
axes[1].set_title('Log-Likelihood')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"MLE: p = {np.sum(data) / len(data)}")

---

## Oefening 2: MLE voor Normale Verdeling - Oplossingen

In [ ]:
# Opdracht 2a - Analytische MLE
data = np.array([2.3, 4.1, 3.5, 5.2, 3.8, 4.6, 3.2, 4.8, 3.9, 4.2])

# Analytische oplossingen:
# μ_MLE = (1/n) Σ x_i = sample mean
# σ²_MLE = (1/n) Σ (x_i - μ)² = sample variance (met n, niet n-1)

mu_mle = np.mean(data)
sigma2_mle = np.mean((data - mu_mle)**2)  # Let op: /n niet /(n-1)
sigma_mle = np.sqrt(sigma2_mle)

print(f"Data: {data}")
print(f"\nAnalytische MLE:")
print(f"  μ_MLE = {mu_mle:.4f}")
print(f"  σ²_MLE = {sigma2_mle:.4f}")
print(f"  σ_MLE = {sigma_mle:.4f}")

In [ ]:
# Opdracht 2b - Numerieke verificatie
def neg_log_likelihood_normal(params, data):
    mu, sigma = params
    if sigma <= 0:
        return np.inf
    n = len(data)
    nll = n/2 * np.log(2*np.pi) + n * np.log(sigma) + np.sum((data - mu)**2) / (2*sigma**2)
    return nll

result = minimize(neg_log_likelihood_normal, x0=[0, 1], args=(data,), method='Nelder-Mead')
mu_num, sigma_num = result.x

print(f"Numerieke optimalisatie:")
print(f"  μ = {mu_num:.4f}")
print(f"  σ = {sigma_num:.4f}")
print(f"\nVerschil met analytisch:")
print(f"  |μ_num - μ_anal| = {abs(mu_num - mu_mle):.6f}")
print(f"  |σ_num - σ_anal| = {abs(sigma_num - sigma_mle):.6f}")

---

## Oefening 3: MSE = Gaussian MLE - Oplossingen

In [ ]:
# Opdracht 3a - Bewijs dat MSE en Gaussian NLL hetzelfde minimum geven

# Genereer data
np.random.seed(42)
n = 30
x = np.linspace(0, 10, n)
true_w, true_b = 2, 1
y = true_w * x + true_b + np.random.normal(0, 1, n)

def mse_loss(params, x, y):
    w, b = params
    pred = w * x + b
    return np.mean((y - pred)**2)

def gaussian_nll(params, x, y, sigma=1):
    w, b = params
    pred = w * x + b
    n = len(y)
    nll = n/2 * np.log(2*np.pi*sigma**2) + np.sum((y - pred)**2) / (2*sigma**2)
    return nll

# Optimaliseer beide
result_mse = minimize(mse_loss, x0=[0, 0], args=(x, y))
result_nll = minimize(gaussian_nll, x0=[0, 0], args=(x, y))

print(f"True: w = {true_w}, b = {true_b}")
print(f"\nMSE optimum: w = {result_mse.x[0]:.4f}, b = {result_mse.x[1]:.4f}")
print(f"NLL optimum: w = {result_nll.x[0]:.4f}, b = {result_nll.x[1]:.4f}")
print(f"\nZe zijn identiek omdat MSE ∝ Gaussian NLL!")

---

## Oefening 4: Cross-Entropy = Categorical MLE - Oplossingen

In [ ]:
# Opdracht 4a - Bewijs equivalentie

def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=-1, keepdims=True))
    return exp_z / np.sum(exp_z, axis=-1, keepdims=True)

def cross_entropy_loss(logits, y_true):
    """Cross-entropy: -Σ y_k * log(p_k)"""
    probs = softmax(logits)
    n = len(y_true)
    return -np.mean(np.log(probs[np.arange(n), y_true] + 1e-10))

def categorical_nll(logits, y_true):
    """NLL: -Σ log p(y_i | x_i)"""
    probs = softmax(logits)
    n = len(y_true)
    # p(y | x) voor categorische verdeling = p_k waar k de juiste klasse is
    return -np.mean(np.log(probs[np.arange(n), y_true] + 1e-10))

# Test
np.random.seed(42)
logits = np.random.randn(5, 3)  # 5 samples, 3 klassen
y_true = np.array([0, 1, 2, 1, 0])

ce = cross_entropy_loss(logits, y_true)
nll = categorical_nll(logits, y_true)

print(f"Cross-Entropy: {ce:.6f}")
print(f"Categorical NLL: {nll:.6f}")
print(f"\nZe zijn identiek!")

---

## Oefening 5: MLE voor Classifier - Oplossingen

In [ ]:
# Opdracht 5a - Logistische regressie via MLE
from sklearn.datasets import make_classification

# Data
X, y = make_classification(n_samples=200, n_features=2, n_redundant=0,
                           n_informative=2, n_clusters_per_class=1,
                           random_state=42)

class LogisticRegressionMLE:
    def __init__(self):
        self.W = None
        self.b = None
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-np.clip(z, -500, 500)))
    
    def negative_log_likelihood(self, params, X, y):
        W = params[:-1]
        b = params[-1]
        z = X @ W + b
        p = self.sigmoid(z)
        # NLL = -Σ [y*log(p) + (1-y)*log(1-p)]
        nll = -np.mean(y * np.log(p + 1e-10) + (1-y) * np.log(1-p + 1e-10))
        return nll
    
    def fit(self, X, y):
        n_features = X.shape[1]
        initial = np.zeros(n_features + 1)
        result = minimize(self.negative_log_likelihood, initial, args=(X, y), method='BFGS')
        self.W = result.x[:-1]
        self.b = result.x[-1]
        return self
    
    def predict_proba(self, X):
        return self.sigmoid(X @ self.W + self.b)
    
    def predict(self, X):
        return (self.predict_proba(X) > 0.5).astype(int)

# Train
model = LogisticRegressionMLE()
model.fit(X, y)

accuracy = np.mean(model.predict(X) == y)
print(f"Training accuracy: {accuracy:.4f}")
print(f"Parameters: W = {model.W}, b = {model.b:.4f}")

In [ ]:
# Visualiseer decision boundary
plt.figure(figsize=(10, 6))

xx, yy = np.meshgrid(np.linspace(X[:,0].min()-1, X[:,0].max()+1, 100),
                     np.linspace(X[:,1].min()-1, X[:,1].max()+1, 100))
grid = np.c_[xx.ravel(), yy.ravel()]
Z = model.predict_proba(grid).reshape(xx.shape)

plt.contourf(xx, yy, Z, levels=20, cmap='RdBu', alpha=0.7)
plt.colorbar(label='P(y=1)')
plt.scatter(X[y==0, 0], X[y==0, 1], c='blue', edgecolors='k', label='Class 0')
plt.scatter(X[y==1, 0], X[y==1, 1], c='red', edgecolors='k', label='Class 1')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.title('Logistische Regressie via MLE')
plt.legend()
plt.show()

---

## Oefening 6: L2 Regularisatie als Prior - Oplossingen

In [ ]:
# Opdracht 6a - Ridge regressie = MAP met Gaussian prior

# Data met noise
np.random.seed(42)
n = 15
x = np.linspace(0, 1, n)
y_true = np.sin(2 * np.pi * x)
y = y_true + np.random.normal(0, 0.3, n)

# Polynomiale features
degree = 9
X = np.column_stack([x**i for i in range(degree + 1)])

def ridge_map(X, y, lam):
    """MAP met Gaussian prior = Ridge regression."""
    n_features = X.shape[1]
    I = np.eye(n_features)
    I[0, 0] = 0  # Don't regularize bias
    w = np.linalg.solve(X.T @ X + lam * I, X.T @ y)
    return w

# Vergelijk verschillende λ
lambdas = [0, 1e-6, 1e-3, 1]
x_plot = np.linspace(0, 1, 100)
X_plot = np.column_stack([x_plot**i for i in range(degree + 1)])

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

for ax, lam in zip(axes.flatten(), lambdas):
    w = ridge_map(X, y, lam)
    y_pred = X_plot @ w
    
    ax.scatter(x, y, color='blue', s=50, label='Data')
    ax.plot(x_plot, y_pred, 'r-', linewidth=2, label='Fit')
    ax.plot(x_plot, np.sin(2*np.pi*x_plot), 'g--', alpha=0.5, label='True')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_title(f'λ = {lam}')
    ax.set_ylim(-2, 2)
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)

plt.suptitle('Ridge Regression = MAP met Gaussian Prior', fontsize=14)
plt.tight_layout()
plt.show()

---

## Oefening 7: MLE vs MAP - Oplossingen

In [ ]:
# Opdracht 7a - Vergelijk MLE en MAP

# Klein dataset → overfitting risico
np.random.seed(42)
n_train = 10
n_test = 100

x_train = np.random.uniform(0, 1, n_train)
y_train = np.sin(2*np.pi*x_train) + np.random.normal(0, 0.3, n_train)

x_test = np.linspace(0, 1, n_test)
y_test = np.sin(2*np.pi*x_test)

degree = 9
X_train = np.column_stack([x_train**i for i in range(degree + 1)])
X_test = np.column_stack([x_test**i for i in range(degree + 1)])

# MLE (λ = 0)
w_mle = np.linalg.lstsq(X_train, y_train, rcond=None)[0]
y_mle = X_test @ w_mle
mse_mle = np.mean((y_test - y_mle)**2)

# MAP (λ = 0.01)
lam = 0.01
I = np.eye(degree + 1)
I[0, 0] = 0
w_map = np.linalg.solve(X_train.T @ X_train + lam * I, X_train.T @ y_train)
y_map = X_test @ w_map
mse_map = np.mean((y_test - y_map)**2)

print(f"Test MSE (MLE): {mse_mle:.4f}")
print(f"Test MSE (MAP): {mse_map:.4f}")
print(f"\nMAP is beter door regularisatie!")

# Plot
plt.figure(figsize=(10, 6))
plt.scatter(x_train, y_train, color='blue', s=80, label='Training data', zorder=5)
plt.plot(x_test, y_test, 'g--', linewidth=2, label='True function')
plt.plot(x_test, y_mle, 'r-', linewidth=2, label=f'MLE (MSE={mse_mle:.3f})')
plt.plot(x_test, y_map, 'b-', linewidth=2, label=f'MAP (MSE={mse_map:.3f})')
plt.xlabel('x')
plt.ylabel('y')
plt.title('MLE vs MAP: Regularisatie voorkomt overfitting')
plt.legend()
plt.ylim(-2, 2)
plt.grid(True, alpha=0.3)
plt.show()

---

## Bonusoefening: Fisher Information - Oplossingen

In [ ]:
# Bonus - Fisher information voor Bernoulli
# I(p) = E[(d/dp log p(x|p))²] = 1 / (p(1-p))

# De variance van de MLE schatter is asymptotisch 1/I(p)
# Dus Var(p_MLE) ≈ p(1-p) / n

def fisher_information_bernoulli(p):
    return 1 / (p * (1 - p))

# Simuleer om te verifiëren
true_p = 0.7
sample_sizes = [10, 50, 100, 500, 1000]
n_experiments = 1000

print("Verificatie: Var(p_MLE) ≈ p(1-p)/n")
print(f"True p = {true_p}")
print(f"Theoretical factor p(1-p) = {true_p * (1-true_p):.4f}")
print()

for n in sample_sizes:
    estimates = []
    for _ in range(n_experiments):
        data = np.random.binomial(1, true_p, n)
        p_mle = np.mean(data)
        estimates.append(p_mle)
    
    var_empirical = np.var(estimates)
    var_theoretical = true_p * (1 - true_p) / n
    
    print(f"n={n:4d}: Var(empirical)={var_empirical:.6f}, Var(theory)={var_theoretical:.6f}")

In [ ]:
# Visualiseer Fisher information
p_values = np.linspace(0.01, 0.99, 100)
fisher_info = fisher_information_bernoulli(p_values)

plt.figure(figsize=(10, 5))
plt.plot(p_values, fisher_info, 'b-', linewidth=2)
plt.xlabel('p', fontsize=12)
plt.ylabel('Fisher Information I(p)', fontsize=12)
plt.title('Fisher Information voor Bernoulli: I(p) = 1/(p(1-p))', fontsize=14)
plt.grid(True, alpha=0.3)
plt.show()

print("Fisher information is hoog bij p ≈ 0 of p ≈ 1")
print("Dit betekent dat de MLE schatter daar preciezer is.")
print("Bij p = 0.5 is Fisher info minimaal → meeste onzekerheid.")

---

**Mathematical Foundations** | Les 10 Oplossingen | IT & Artificial Intelligence

---